In [3]:
#!pwsh
function ConvertFrom-MdTable {param([string]$MdTable)
    $lines = $MdTable -split "`r?`n"
    $headers = ($lines[0] -split '\|') | ForEach-Object { $_.Trim() } | Where-Object { $_ }
    $lines[2..($lines.Length - 1)] | ForEach-Object {
        $cells = ($_ -split '\|') | ForEach-Object { $_.Trim() } | Where-Object { $_ }
        $obj = @{}
        for ($i = 0; $i -lt $headers.Count; $i++) {
            $obj[$headers[$i]] = $cells[$i]
        }
        [PSCustomObject]$obj
    }
}

In [ ]:
#$Host.UI.RawUI.BufferSize = New-Object Management.Automation.Host.Size(127, $Host.UI.RawUI.BufferSize.Height)
Get-AzContext

In [ ]:
# Markdown table with each VM parameters in a row
$table = @"
| VMName   | Size              | Subnet  | VNet       | Image                                                   |
|----------|-------------------|---------|------------|---------------------------------------------------------|
| TestVm1  | Standard_D2as_v5  | default | TestVmVn   | MicrosoftWindowsDesktop:Windows-11:win11-23h2-ent:latest |
| TestVm2  | Standard_D2as_v5  | default | TestVmVn   | MicrosoftWindowsDesktop:Windows-11:win11-24h2-ent:latest |
"@

$VMs = ConvertFrom-MdTable -MdTable $Table
$VMs | Format-Table


Image                                                    VMName  Subnet  VNet     Size
-----                                                    ------  ------  ----     ----
MicrosoftWindowsDesktop:Windows-11:win11-23h2-ent:latest TestVm1 default TestVmVn Standard_D2as_v5
MicrosoftWindowsDesktop:Windows-11:win11-24h2-ent:latest TestVm2 default TestVmVn Standard_D2as_v5



In [ ]:
$AzParams     = @{Location = 'NorthCentralUS'; ResourceGroupName  = 'TestVmRg';  AllocationMethod = 'Static'}
$DeleteOption = @{NetworkInterfaceDeleteOption = 'Delete';OSDiskDeleteOption = 'Delete'; DataDiskDeleteOption = 'Delete'}
$cred         = New-Object System.Management.Automation.PSCredential "<>",$(ConvertTo-SecureString '<>' -asplaintext -force)

$VMs | ForEach-Object {
    New-AzVM @AzParams @DeleteOption -Name $_.VMName -Size $_.Size -Image $_.Image -Credential $cred -SubnetName $_.Subnet -VirtualNetworkName $_.VNet
}


You can reference https://aka.ms/findImagePS on how to find VM Images using PowerShell.




ResourceGroupName        : TestVmRg
Id                       : /subscriptions/<>/resourceGroups/TestVmRg/providers/Microsoft.Com
pute/virtualMachines/TestVm1
VmId                     : e8a78f01-7e85-4ce4-833e-e23644c7b12a
Name                     : TestVm1
Type                     : Microsoft.Compute/virtualMachines
Location                 : northcentralus
Tags                     : {}
HardwareProfile          : {VmSize}
NetworkProfile           : {NetworkInterfaces}
OSProfile                : {ComputerName, AdminUsername, WindowsConfiguration, Secrets, AllowExtensionOperations, 
RequireGuestProvisionSignal}
ProvisioningState        : Succeeded
StorageProfile           : {ImageReference, OsDisk, DataDisks, DiskControllerType}
FullyQualifiedDomainName : testvm1-5cf25b.NorthCentralUS.cloudapp.azure.com
TimeCreated              : 4/2/2025 3:18:00 AM
Etag                     : "2"

You can reference https://aka.ms/findImagePS on how to find VM Images using PowerShell.

ResourceGroupName